<a href="https://colab.research.google.com/github/CaseySobon/Recommendation-system/blob/main/ALS_fail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

In [ ]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

from google.colab import files

In [ ]:
uploaded = files.upload()

Saving test2_new.txt to test2_new.txt


In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


spark = SparkSession.builder.appName('recommend-ML').getOrCreate()

df = spark.read.csv('test2_new.txt', header = False, inferSchema = True,sep='|')
cols = df.columns
sparkDF=df.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "trackID").withColumnRenamed("_c2", "prediction")
sparkDF.printSchema()
sparkDF.show(5)

root
 |-- userID: integer (nullable = true)
 |-- trackID: integer (nullable = true)
 |-- prediction: integer (nullable = true)

+------+-------+----------+
|userID|trackID|prediction|
+------+-------+----------+
|200031|  30877|         1|
|200031|   8244|         1|
|200031| 130183|         0|
|200031| 198762|         0|
|200031|  34503|         1|
+------+-------+----------+
only showing top 5 rows



In [ ]:
x1 = pd.read_csv('testTrack_hierarchy.txt', sep='|', header=None, names=range(14),na_values='None',dtype=str).values

x2=pd.DataFrame(x1)
x3=x2.fillna(0)
x3[:] = x3[:].astype(int)
H_DF=spark.createDataFrame(x3).withColumnRenamed("0", "userID").withColumnRenamed("1", "trackID").withColumnRenamed("2", "itemID1")\
.withColumnRenamed("3", "itemID2").withColumnRenamed("4", "itemID3").withColumnRenamed("5", "itemID4").withColumnRenamed("6", "itemID5")\
.withColumnRenamed("7", "itemID6").withColumnRenamed("8", "itemID7").withColumnRenamed("9", "itemID8").withColumnRenamed("10", "itemID9")\
.withColumnRenamed("11", "itemID10").withColumnRenamed("12", "itemID11").withColumnRenamed("13", "itemID12")


In [ ]:
H_DF.show(5)

+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+
|userID|trackID|itemID1|itemID2|itemID3|itemID4|itemID5|itemID6|itemID7|itemID8|itemID9|itemID10|itemID11|itemID12|
+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+
|199810| 208019| 209288|      0|      0|      0|      0|      0|      0|      0|      0|       0|       0|       0|
|199810|  74139| 277282| 271146| 113360| 173467| 173655| 192976| 146792|  48505| 133159|       0|       0|       0|
|199810|   9903|      0|      0|  33722| 123396|  79926|  73523|      0|      0|      0|       0|       0|       0|
|199810| 242681| 190640| 244574|  61215|  17453| 274088|      0|      0|      0|      0|       0|       0|       0|
|199810|  18515| 146344|  33168|  19913|  48505| 154024|      0|      0|      0|      0|       0|       0|       0|
+------+-------+-------+-------+-------+-------+-------+-------+-------+

In [ ]:
from pyspark.sql.functions import array, col, explode, lit, struct
from pyspark.sql import DataFrame
from typing import Iterable
def melt(H_DF: DataFrame, id_vars: Iterable[str], value_vars: Iterable[str], var_name: str="variable", value_name: str="value") -> DataFrame:
    """Convert :class:`DataFrame` from wide to long format."""

    # Create array<struct<variable: str, value: ...>>
    _vars_and_vals = array(*(struct(lit(c).alias(var_name), col(c).alias(value_name)) for c in value_vars))

    # Add to the DataFrame and explode
    _tmp = H_DF.withColumn("_vars_and_vals", explode(_vars_and_vals))

    cols = id_vars + [col("_vars_and_vals")[x].alias(x) for x in [var_name, value_name]]
    return _tmp.select(*cols)

sparkP=melt(H_DF,id_vars=['userID','trackID'],value_vars=['itemID1', 'itemID2','itemID3',\
                                                            'itemID4','itemID5', 'itemID6',\
                                                            'itemID7', 'itemID8','itemID9',\
                                                            'itemID10', 'itemID11','itemID12'])

sparkP.show(5)

+------+-------+--------+------+
|userID|trackID|variable| value|
+------+-------+--------+------+
|199810| 208019| itemID1|209288|
|199810| 208019| itemID2|     0|
|199810| 208019| itemID3|     0|
|199810| 208019| itemID4|     0|
|199810| 208019| itemID5|     0|
+------+-------+--------+------+
only showing top 5 rows



In [ ]:
df2=sparkP.where("value!=0")
df3=df2.withColumnRenamed("value", "itemID")

print((df3.count(), len(df3.columns)))
print((H_DF.count(), len(H_DF.columns)))
df3.show(5)

(630530, 4)
(120000, 14)
+------+-------+--------+------+
|userID|trackID|variable|itemID|
+------+-------+--------+------+
|199810| 208019| itemID1|209288|
|199810|  74139| itemID1|277282|
|199810|  74139| itemID2|271146|
|199810|  74139| itemID3|113360|
|199810|  74139| itemID4|173467|
+------+-------+--------+------+
only showing top 5 rows



In [ ]:
from pyspark.sql.types import IntegerType

ratings = spark.read.csv("trainIdx2_matrix.txt", header = False,sep='|')
ratings=ratings.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "rating")

ratings = ratings.withColumn("userID", ratings["userID"].cast(IntegerType()))
ratings = ratings.withColumn("itemID", ratings["itemID"].cast(IntegerType()))
ratings = ratings.withColumn("rating", ratings["rating"].cast('float'))

ratings.printSchema()
ratings.show(5)

root
 |-- userID: integer (nullable = true)
 |-- itemID: integer (nullable = true)
 |-- rating: float (nullable = true)

+------+------+------+
|userID|itemID|rating|
+------+------+------+
|199808|248969|  90.0|
|199808|  2663|  90.0|
|199808| 28341|  90.0|
|199808| 42563|  90.0|
|199808| 59092|  90.0|
+------+------+------+
only showing top 5 rows



In [ ]:
from pyspark.sql import functions as f
from pyspark.sql.functions import when,concat_ws

merged_rating = df3.join(ratings, ['userID','itemID'],'left').orderBy('userID', ascending=True).fillna(0)
merged_rating.show(50)


+------+------+-------+--------+------+
|userID|itemID|trackID|variable|rating|
+------+------+-------+--------+------+
|199810|190640| 242681| itemID1|   0.0|
|199810|154024|  18515| itemID5|   0.0|
|199810|173655|  74139| itemID5|   0.0|
|199810| 11616| 105760| itemID2|  90.0|
|199810| 73523|   9903| itemID6|   0.0|
|199810|113360|  74139| itemID3|   0.0|
|199810| 48505|  18515| itemID4|   0.0|
|199810| 48505| 105760| itemID5|   0.0|
|199810|173467| 105760| itemID4|  80.0|
|199810|209288| 208019| itemID1|   0.0|
|199810| 93458| 105760| itemID1|   0.0|
|199810|244574| 242681| itemID2|   0.0|
|199810| 48505|  74139| itemID8|   0.0|
|199810|271146|  74139| itemID2|   0.0|
|199810|131552| 105760| itemID3|  80.0|
|199810|277282|  74139| itemID1|   0.0|
|199810|146344|  18515| itemID1|   0.0|
|199810|133159|  74139| itemID9|   0.0|
|199810| 17453| 242681| itemID4|   0.0|
|199810| 33722|   9903| itemID3|   0.0|
|199810|192976|  74139| itemID6|   0.0|
|199810|133159| 105760| itemID6|   0.0|


In [ ]:
print((merged_rating.count(), len(merged_rating.columns)))
merged_rating.show(50)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

als = ALS(
    maxIter=10,
    rank = 10,
    regParam=0.01,
    userCol="userID",
    itemCol="itemID",
    ratingCol="rating",
    nonnegative = True,
    implicitPrefs = False,
    coldStartStrategy="nan"
)
model = als.fit(merged_rating)
predictions = model.transform(merged_rating)
predictions.show(50)

+------+------+-------+--------+------+-----------+
|userID|itemID|trackID|variable|rating| prediction|
+------+------+-------+--------+------+-----------+
|199810| 11616| 105760| itemID2|  90.0|      79.95|
|199810| 17453| 242681| itemID4|   0.0|   8.812636|
|199810| 33168|  18515| itemID2|  70.0|  45.179405|
|199810| 33722|   9903| itemID3|   0.0|  28.860617|
|199810| 61215| 242681| itemID3|   0.0|   4.389797|
|199810| 79926|   9903| itemID5|   0.0|  0.1998515|
|199810| 93458| 105760| itemID1|   0.0|  22.856178|
|199810|131552| 105760| itemID3|  80.0|   62.60923|
|199810|133159|  74139| itemID9|   0.0|  0.7684499|
|199810|133159| 105760| itemID6|   0.0|  0.7684499|
|199810|146792|  74139| itemID7|   0.0| 0.34596816|
|199810|173467|  74139| itemID4|  80.0|  81.507545|
|199810|173467| 105760| itemID4|  80.0|  81.507545|
|199810|209288| 208019| itemID1|   0.0|  3.6575308|
|199810|244574| 242681| itemID2|   0.0|  10.383484|
|199810|271146|  74139| itemID2|   0.0|   27.40323|
|199810|2772

In [ ]:
H_DF= H_DF.select('userID','trackID')
print((H_DF.count(), len(H_DF.columns)))


(120000, 2)


In [ ]:
predict1=predictions.groupBy('userID','trackID',).max('prediction').orderBy('userID')
predict1.show(5)
predict1=H_DF.join(predict1, ['userID','trackID'],'left').orderBy('userID').fillna(0)
print((predict1.count(), len(predict1.columns)))


+------+-------+---------------+
|userID|trackID|max(prediction)|
+------+-------+---------------+
|199810|  18515|      45.179405|
|199810|  74139|      81.507545|
|199810| 208019|      3.6575308|
|199810| 242681|      10.383484|
|199810| 105760|      81.507545|
+------+-------+---------------+
only showing top 5 rows

(120000, 3)


In [ ]:
from pyspark.sql import functions as f
from pyspark.sql.functions import when,concat_ws
from pyspark.sql.window import *
from pyspark.sql.functions import row_number

predict_f=predict1.withColumn("row_number", f.row_number().over(Window.partitionBy("userID").orderBy("max(prediction)")))
predict_f1=predict_f.withColumnRenamed("row_number","Predictor")

predict_f1.show(6)
predict_f1=predict_f1.withColumn("Predictor", when(predict_f1.Predictor >= 4,1).otherwise(0))
predict_f1=predict_f1.select(concat_ws('_',predict_f1.userID,predict_f1.trackID).alias("TrackID"),'Predictor')
predict_f1.show(6)
print((predict_f1.count(), len(predict_f1.columns)))

+------+-------+---------------+---------+
|userID|trackID|max(prediction)|Predictor|
+------+-------+---------------+---------+
|199810| 208019|      3.6575308|        1|
|199810| 242681|      10.383484|        2|
|199810|   9903|      28.860617|        3|
|199810|  18515|      45.179405|        4|
|199810|  74139|      81.507545|        5|
|199810| 105760|      81.507545|        6|
+------+-------+---------------+---------+
only showing top 6 rows

+-------------+---------+
|      TrackID|Predictor|
+-------------+---------+
|199810_208019|        0|
|199810_242681|        0|
|  199810_9903|        0|
| 199810_18515|        1|
| 199810_74139|        1|
|199810_105760|        1|
+-------------+---------+
only showing top 6 rows

(120000, 2)


In [ ]:
predict_f1.toPandas().to_csv('myprediction_sprk3.csv',index=False)

In [ ]:
merged=merged_rating.groupBy('userID','trackID',).max('rating').orderBy('userID')
merged=merged.withColumnRenamed("max(rating)","Predictor")
merged.show(16)
print((merged.count(), len(merged.columns)))

+------+-------+---------+
|userID|trackID|Predictor|
+------+-------+---------+
|199810|  74139|     80.0|
|199810| 105760|     90.0|
|199810|  18515|     70.0|
|199810| 208019|      0.0|
|199810| 242681|      0.0|
|199810|   9903|      0.0|
|199812| 211361|      0.0|
|199812| 276940|      0.0|
|199812| 130023|    100.0|
|199812| 223706|    100.0|
|199812|  29189|     80.0|
|199812| 142408|    100.0|
|199813|  20968|     80.0|
|199813|  21571|     90.0|
|199813|  79640|     90.0|
|199813| 184173|     80.0|
+------+-------+---------+
only showing top 16 rows

(119025, 3)


In [ ]:
merged_complete=H_DF.join(merged, ['userID','trackID'],'left').orderBy('userID').fillna(0)
merged_complete.show(16)

+------+-------+---------+
|userID|trackID|Predictor|
+------+-------+---------+
|199810| 208019|      0.0|
|199810| 242681|      0.0|
|199810| 105760|     90.0|
|199810|   9903|      0.0|
|199810|  74139|     80.0|
|199810|  18515|     70.0|
|199812| 130023|    100.0|
|199812|  29189|     80.0|
|199812| 211361|      0.0|
|199812| 223706|    100.0|
|199812| 276940|      0.0|
|199812| 142408|    100.0|
|199813|  20968|     80.0|
|199813| 184173|     80.0|
|199813| 111874|     80.0|
|199813|  79640|     90.0|
+------+-------+---------+
only showing top 16 rows



In [ ]:
merged_complete.toPandas().to_csv('merged_raw.csv',index=False)